# Entregable 1: Archivo de lectura de Datos Oficiales y Datos Blind

## Importar Datos desde Sitio Web de la ANH (Datos Oficiales)

In [1]:
import os
import pandas as pd
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
import time
from multiprocessing.pool import ThreadPool
from progress.bar import ChargingBar

#*******************************************************************************************
# Crear Clase para Importar datos desde pagina web de la ANH
class Importar_Datos_Oficiales:


    def __init__(self):
        
        #****************************************************
        # Problema al Descargar datos desde Pagina web de la ANH:
        # Url: http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents
        # Mensaje de Error: "RemoteDisconnected: Remote end closed connection without response"
        # Modificacion: Se importa los archivos directamente del repositorio SPEcolombiahackathon
        #*****************************************************

        
        BASE_URL = 'https://github.com/specolombiahackathon/202010/blob/main/' #url del repositorio (incluir raw)
        # Url's de los archivos de produccion de crudo:
        file2013_url = 'Producci%C3%B3n%20fiscalizada%20de%20crudo%20a%C3%B1o%202013.xlsx?raw=true'
        file2014_url = 'Producci%C3%B3n%20fiscalizada%20de%20crudo_2014_31122014.xlsx?raw=true'
        file2015_url = 'Producci%C3%B3n%20Fiscalizada%20Crudo%202015.xlsx?raw=true'
        file2016_url = 'Producci%C3%B3n%20fiscalizada%20crudo%202016_18042018.xls?raw=true'
        file2017_url = 'Produccion-fiscalizada-crudo-2017.xlsx?raw=true'
        file2018_url = 'Producci%c3%b3n%20Fiscalizada%20Crudo%202018.xlsx?raw=true' 
        file2019_url = 'Producci%c3%b3n%20Fiscalizada%20Crudo%202019-DIC.xlsx?raw=true'
        file2020_url = 'Producci%c3%b3n%20Fiscalizada%20Crudo%202020%20Agosto.xlsx?raw=true'
        
        self.URLS = {'Produccion_Fiscalizada_Crudo_2018': f'{BASE_URL}'+file2018_url,
                'Produccion_Fiscalizada_Crudo_2019': f'{BASE_URL}'+file2019_url,
                'Produccion_Fiscalizada_Crudo_2020':f'{BASE_URL}'+file2020_url,
                'Produccion_Fiscalizada_Crudo_2017':f'{BASE_URL}'+file2017_url,
                'Produccion_Fiscalizada_Crudo_2016':f'{BASE_URL}'+file2016_url,
                'Produccion_Fiscalizada_Crudo_2015':f'{BASE_URL}'+file2015_url,
                'Produccion_Fiscalizada_Crudo_2014':f'{BASE_URL}'+file2014_url,
                'Produccion_Fiscalizada_Crudo_2013':f'{BASE_URL}'+file2013_url,
        }


        self.data = {case:pd.read_excel(url,usecols = 'A:Q') for case, url in self.URLS.items()}
#*******************************************************************************************

# Obtener datos desde la pagina web de la ANH
datos_importados = Importar_Datos_Oficiales()

# Guardar datos importados en DataFrames para posterior procesamiento
df_2013 = datos_importados.data['Produccion_Fiscalizada_Crudo_2013']
df_2014 = datos_importados.data['Produccion_Fiscalizada_Crudo_2014']
df_2015 = datos_importados.data['Produccion_Fiscalizada_Crudo_2015']
df_2016 = datos_importados.data['Produccion_Fiscalizada_Crudo_2016']
df_2017 = datos_importados.data['Produccion_Fiscalizada_Crudo_2017']
df_2018 = datos_importados.data['Produccion_Fiscalizada_Crudo_2018']
df_2019 = datos_importados.data['Produccion_Fiscalizada_Crudo_2019']
df_2020 = datos_importados.data['Produccion_Fiscalizada_Crudo_2020']

In [2]:
df_2020.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,AGENCIA NACIONAL DE HIDROCARBUROS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"VICEPRESIDENCIA DE OPERACIONES, REGALIAS Y PAR...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRODUCCION FISCALIZADA DE PETROLEO POR CAMPO (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,enero-agosto de 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Departamento,Municipio,Operadora,Contrato,Campo,enero,febrero,marzo,abril,mayo,junio,julio,agosto
6,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949
7,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08
8,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73
9,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94


In [3]:
# Funcion para limpieza del archivo excel y poner en formato
def format_excel(df_excel_raw, year = 2020):
    df_excel_raw.dropna(inplace = True) #eliminar filas NO validas
    if np.any(np.char.find(df_excel_raw.columns.values.astype(str),'Unnamed:') == 0 ): # Verificar si encabezado tiene algun elemento 'Unnamed' (Nota: cambio en codigo por ENCABEZADO)
        df_excel_raw.rename(columns=df_excel_raw.iloc[0],inplace = True) #reemplazar el nombre de las columnas si no tienen nombre ('Unnamed')
        df_excel_raw.drop(df_excel_raw.index[0], inplace = True)
        df_excel_raw.reset_index(drop = True, inplace = True) # reiniciar el indexado de las filas
    df_excel_raw.columns = df_excel_raw.columns.str.strip().str.lower() # convertir nombre de columnas a minusculas y sin espacios
    
    meses = ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
        
    for mes in meses:
        if (0 in np.core.defchararray.find( df_excel_raw.columns.values.astype(str), mes)):
            # reemplazar nombre de Mes a nombre corto (Enero --> ene)
            df_excel_raw.columns.values[[i for i,item in enumerate(df_excel_raw.columns.values) if mes in item]] = mes            

        else:
            # Si no hay datos para el mes, anadir una columna de ceros para ese mes
            df_excel_raw[mes] = pd.Series([0 for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
            
    # anadir columna con año para identificacion
    df_excel_raw['año'] = pd.Series([year for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
    
    return df_excel_raw

In [4]:
format_excel(df_2020,2020)
format_excel(df_2019,2019)
format_excel(df_2018,2018)
format_excel(df_2017,2017)
format_excel(df_2016,2016)
format_excel(df_2015,2015)
format_excel(df_2014,2014)
format_excel(df_2013,2013)

,campo,contrato,cuenca,empresa,departamento,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,Abanico,Abanico,Valle Superior del Magdalena,PACIFIC STRATUS ENERGY,Tolima,1220.71,1258.89,1248.9,1220.83,1194.68,1112.8,1014.35,995.258,1014.6,1142.32,1148.1,1160.1,2013
1,Abarco,Nare,Valle Medio del Magdalena,MANSAROVAR ENERGY COLOMBIA LTD.,Boyacá,5027.19,5051.25,5700.42,5654.4,5240.03,5066.3,5580.65,5647.94,5562.07,5959.61,6533.97,6279.42,2013
2,Abedus,Cravoviejo,Llanos Orientales,C&C ENERGÍA,Casanare,0,0,0,33.6667,3.90323,0.233333,5.06452,19,0,0,0,0,2013
3,Abejas,Estero,Llanos Orientales,PERENCO COLOMBIA LIMITED,Casanare,233.806,307.929,262.968,265.933,271.484,276.533,278.806,272.226,290.633,136.903,0,79.3226,2013
4,Acacia Este,Las Quinchas,Valle Medio del Magdalena,PACIFIC STRATUS ENERGY,Santander,0,0,3.51613,4.36667,4.64516,4.96667,4.29032,4.74194,3.86667,2.74194,4.43333,4.83871,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Yenac,Casimena,Llanos Orientales,PETROMINERALES COLOMBIA LTD.,Casanare,1641.81,1717.39,1648.42,1594.97,1589,1217.63,1455.06,1476.35,1351.37,1292.35,1324.2,1330.68,2013
420,Yopo,Cubiro,Llanos Orientales,ALANGE ENERGY,Casanare,1132.23,823.393,819.613,667.967,702.677,638.767,569.323,549.355,442.233,522.323,611.2,608.323,2013
421,Yurilla,Mansoyá,Putumayo,ECOPETROL S.A.,Putumayo,232.677,221.643,180.161,208.7,294.71,316.467,281.355,167.548,113.2,278.194,44.7333,0,2013
422,Zoe,Midas,Valle Medio del Magdalena,ECOPETROL S.A.,Cesar,0,0,0,0,10.2903,12.2333,2,6.67742,0,0,2.7,8.06452,2013


In [5]:
df_2017.head(5) #mostrar produccion de crudo - 2020

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,1230.97,1108.93,1026.16,1035.77,1046.87,1122.08,1133.03,1163.72,1130.25,1077.69,1156.24,1099.28,2017
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,NARE,271.742,257.036,252.903,232.9,261.774,216.059,193.717,161.835,231.971,244.219,222.163,208.825,2017
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,UNDERRIVER,935.677,796.929,880.226,899.867,993.742,801.946,765.493,814.503,887.706,845.619,720.345,833.261,2017
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,393,386,361.613,365.433,395.613,381.826,369.09,342.075,279.994,263.75,281.705,277.985,2017
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,CASABE,CASABE,12745,12535.2,12756.4,12720.3,12491.9,13126.9,12376.6,12083.6,11965.4,11750.8,11931.9,11633.8,2017


In [6]:
# Unir produccion desde 2018 a 2020 en Un unico DataFrame
df_2018_2020 = pd.concat([df_2020, df_2019, df_2018], ignore_index = True)
df_2018_2020.tail()

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
1333,TOLIMA,PURIFICACIÓN,HOCOL S.A.,CHENCHE,CHENCHE,49,48.1429,47.2258,47.0667,46.1081,45.213,44.9926,45.0842,38.5367,38.62,47.0023,47.5377,2018
1334,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,MATACHÍN NORTE,1715.71,1676.43,1667.06,1528.73,1406.48,1318.69,1184.35,1311.35,1427.12,1616.3,1545.17,1620.64,2018
1335,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,MATACHÍN SUR,219.613,220,238.129,215.167,215.581,197.589,188.106,171.372,167.229,159.029,135.403,107.891,2018
1336,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,PURIFICACIÓN,504.742,392.893,337.258,299.5,267.742,237.8,221.134,220.36,207.914,202.745,195.347,194.82,2018
1337,TOLIMA,SAN LUIS,HOCOL S.A.,ORTEGA,PACANDE,0,0,0,0,0,0,0,0,0,0,0,78.5519,2018


In [7]:
# convertir datos de produccion a formato numerico
for mes in ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']:
    df_2018_2020[mes] = pd.to_numeric(df_2018_2020[mes])

In [8]:
# Formato LONG: Datos de Produccion de CRUDO (2018 a 2020)
df_2018_2020_LONG = pd.melt(df_2018_2020, id_vars=['departamento','municipio','operadora','contrato','campo','año'], var_name = 'mes', value_name = 'produccion')
df_2018_2020_LONG.head()

,departamento,municipio,operadora,contrato,campo,año,mes,produccion
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,2020,ene,1765.84
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,2020,ene,119.70
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,2020,ene,11636.58
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,2020,ene,1852.60
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,2020,ene,187.33


In [9]:
# Archivo de Salida (Output): Datos de Produccion de Crudo (2018 a 2020):

# Crear Carpeta 'Datos_salida' donde guardar archivos de salida
path = os.getcwd()
print(path)
dir = os.path.join(path,"Datos_salida")
if not os.path.exists(dir):
    os.mkdir(dir)

    
# Exportar a .xlsx - produccion 2018 a 2020
df_2018_2020_LONG.to_excel('Datos_salida/Produccion_Fiscalizada_Crudo_DatosOficiales_2018-2020.xlsx', index = False)

C:\Users\jnina\Jupyter\HackathonOG


## Importar Datos Blind desde Repositorio Hackathon O&G (GitHub)

In [10]:
#*******************************************************************************************
# Crear Clase para Importar datos blind desde Repositorio Hackathon O&G 
class Importar_Datos_Blind:


    def __init__(self):

        BASE_URL = 'https://github.com/specolombiahackathon/202010/blob/main/Data_BlindTest/' #url del repositorio (incluir raw) con datos blind
        # url's de los datos de produccion (blind)
        file2017_url = 'Producci%C3%B3n%20Fiscalizada%20Crudo%202017.xlsx?raw=true'
        file2018_url = 'Producci%C3%B3n%20Fiscalizada%20Crudo_2018_12458.xlsx?raw=true' 
        file2019_url = 'Producci%C3%B3n%20Fiscalizada%20Crudo%20ANH%20Colombia%202019%20-%20final.xlsx?raw=true'
        
        
        self.URLS = {'Produccion_Fiscalizada_Crudo_2018_blind': f'{BASE_URL}'+file2018_url,
                'Produccion_Fiscalizada_Crudo_2019_blind': f'{BASE_URL}'+file2019_url,
                'Produccion_Fiscalizada_Crudo_2017_blind':f'{BASE_URL}'+file2017_url,
                
        }


        self.data = {case:pd.read_excel(url,usecols = 'A:Q') for case, url in self.URLS.items()}
#*******************************************************************************************

# Obtener datos desde la pagina web de la ANH
datos_importados_blind = Importar_Datos_Blind()

# Guardar datos importados en DataFrames para posterior procesamiento
df_2017_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2017_blind']
df_2018_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2018_blind']
df_2019_blind = datos_importados_blind.data['Produccion_Fiscalizada_Crudo_2019_blind']



In [11]:
len(df_2017_blind)

38

In [12]:
format_excel(df_2018_blind,2018)
format_excel(df_2017_blind,2017)
format_excel(df_2019_blind,2019)

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,1f d2689f,7897.993261,6687.201142,6176.648164,5998.336864,5838.969170,6390.660871,4766.241346,5871.527810,5179.292397,5928.184630,6237.904951,6226.606211,2019
1,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,9ac1420f,134.214677,125.917335,123.133429,131.332900,141.304935,139.630242,87.627764,133.442578,107.876324,131.583016,114.477224,114.988332,2019
2,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,9b395bc9,322.628507,259.131545,261.056355,230.248530,149.471782,198.027002,246.364730,306.490557,247.691435,220.330868,193.601028,175.016287,2019
3,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,254d7db5,40.170881,42.411055,38.169949,30.264094,28.524153,29.111383,14.843869,25.751123,30.090100,6.666148,0.000000,0.000000,2019
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,22890.888398,24367.402087,25999.977584,25748.153790,24888.797071,25571.006076,19071.892525,24910.655077,24186.730993,24412.977659,24585.503656,23528.979030,2019
5,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,043b305e,3514.332254,3477.434636,3739.437084,3586.017811,3341.208153,3388.175677,2361.860773,3366.578663,3649.090662,3636.302098,3663.227680,3587.746877,2019
6,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,fca93f9e,26.349227,28.121792,29.394123,27.512813,27.121326,28.600275,15.605093,25.294388,15.550720,14.245765,10.222152,0.000000,2019
7,cf33cb8a,102a28a6,d5580f74,1f d2689f,13276557,205.780613,200.789158,188.479078,157.062273,151.994696,127.776896,67.846813,104.255073,90.009308,114.988332,79.025932,66.541858,2019
8,cf33cb8a,102a28a6,d5580f74,1f d2689f,c220f014,1252.898684,908.227303,665.277207,662.014818,610.208083,516.436153,328.555180,479.397165,375.751072,467.500321,442.271180,433.440981,2019
9,cf33cb8a,102a28a6,d5580f74,1f d2689f,1fd2689f,128.385875,147.155486,148.145077,117.880983,96.262220,97.295310,53.176938,86.649048,76.263776,95.979480,94.500530,95.131259,2019


In [13]:
# Unir produccion desde 2017 a 2019 en Un unico DataFrame
df_2017_2019_blind = pd.concat([df_2019_blind, df_2018_blind, df_2017_blind,], ignore_index = True)
len(df_2017_2019_blind)

122

In [14]:
# convertir datos de produccion a formato numerico
for mes in ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']:
    df_2017_2019_blind[mes] = pd.to_numeric(df_2017_2019_blind[mes])

In [15]:
# Formato LONG: Datos de Produccion de CRUDO (2013 a 2020)
df_2017_2019_LONG_blind = pd.melt(df_2017_2019_blind, id_vars=['departamento','municipio','operadora','contrato','campo','año'], var_name = 'mes', value_name = 'produccion')
df_2017_2019_LONG_blind.tail()

,departamento,municipio,operadora,contrato,campo,año,mes,produccion
1459,f7fd2c4f,e1745f70,ffd6d24d,796c2e32,b111ec69,2017,dic,0.000000
1460,eccb9ef1,870c4a0b,ffd6d24d,a3d02126,27cb99a2,2017,dic,0.000000
1461,0fa93c9b,756c486f,2fe52430,b4dad5fa,48670499,2017,dic,7053.963456
1462,0fa93c9b,756c486f,2fe52430,b4dad5fa,373ebdec,2017,dic,754.001905
1463,0fa93c9b,756c486f,2fe52430,b4dad5fa,e32e23a1,2017,dic,867.614772


In [16]:
# Archivo de Salida (Output): Datos de Produccion de Crudo (2017 a 2019):

# Crear Carpeta 'Datos_salida' donde guardar archivos de salida
path = os.getcwd()
print(path)
dir = os.path.join(path,"Datos_salida")
if not os.path.exists(dir):
    os.mkdir(dir)

    
# Exportar a .xlsx - produccion 2017 a 2019
df_2017_2019_LONG_blind.to_excel('Datos_salida/Produccion_Fiscalizada_Crudo_Blind_2017-2019.xlsx', index = False)

C:\Users\jnina\Jupyter\HackathonOG
